
# NSE Multi‑Ticker Strategy — Rolling Walk‑Forward + ATR Volatility Sizing + Groww‑style Costs

This notebook optimizes a technical strategy for multiple NSE tickers using a rolling walk‑forward process and ATR‑based position sizing (risk per trade as % of equity). It writes:
- `optimized_results_summary.csv`
- `walkforward_params_<TICKER>.csv`
- `trades_<TICKER>.csv`

> Edit the `CONFIG` and `COSTS` blocks to match your setup (especially Groww fees).

In [8]:

# ==========================
# Global Configuration (EDIT)
# ==========================
CONFIG = {
    "tickers": ["RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS"],
    "start_date": "2018-01-01",
    "end_date": None,              # None => latest available
    # Rolling walk-forward
    "wf_train_years": 3,           # years per training window
    "wf_forward_months": 6,        # months per forward window
    "min_train_bars": 252 * 2,     # at least ~2y of train bars
    # Capital & risk
    "initial_capital_per_ticker": 100000.0,  # INR
    "risk_per_trade_pct": 0.01,    # 1% of equity risked per trade
    "compound_across_folds": True, # carry equity across forward folds
    # Slippage (bps per side)
    "slippage_bps": 5,             # 5 bps = 0.05% each side
    # Indicators grid
    "use_ema": False,
    "grid": {
        "fast_ma": [10, 20],
        "slow_ma": [50, 100, 200],
        "rsi_len": [14],
        "rsi_mode": ["momentum", "off"],  # {"momentum","mean_reversion","off"}
        "rsi_mom_min": [55],
        "rsi_mr_max": [35],
        "macd_fast": [12],
        "macd_slow": [26],
        "macd_signal": [9],
        "macd_mode": ["above_signal_for_long", "off"],
        "adx_len": [14],
        "adx_min": [15],
        "atr_len": [14],
        "stop_atr_mult": [2.0, 3.0],
        "tp_rr_multiple": [2.0, 3.0],
        "max_hold_days": [60]
    },
    "objective": "CAGR"  # {"CAGR","TotalReturn","Sharpe"}
}

# ==========================
# India Costs (EDIT to match Groww)
# ==========================
COSTS = {
    "market_mode": "delivery",  # "delivery" or "intraday"
    # Rates use basis points (bps) unless otherwise noted. 1 bps = 0.01%.
    "delivery": {
        "brokerage_bps": 0.0,             # e.g., 5.0 bps for 0.05% if applicable
        "brokerage_cap_per_order": 20.0,  # INR cap per order (set 0 if NA)
        "brokerage_min": 0.0,             # INR minimum brokerage per order
        "stt_buy_bps": 10.0,              # 0.10% buy
        "stt_sell_bps": 10.0,             # 0.10% sell
        "exchange_txn_bps": 0.345,        # ~0.00345%
        "sebi_bps": 0.10,                 # ~10 INR / crore
        "stamp_buy_bps": 1.5,             # e.g., 0.015% buy
        "stamp_sell_bps": 0.0,
        "gst_pct": 18.0                   # % on (brokerage + exchange)
    },
    "intraday": {
        "brokerage_bps": 5.0,             # 0.05% (cap applied below)
        "brokerage_cap_per_order": 20.0,  # INR cap
        "brokerage_min": 0.0,
        "stt_buy_bps": 0.0,
        "stt_sell_bps": 2.5,              # 0.025% sell
        "exchange_txn_bps": 0.345,
        "sebi_bps": 0.10,
        "stamp_buy_bps": 0.3,             # ~0.003% buy
        "stamp_sell_bps": 0.0,
        "gst_pct": 18.0
    }
}

print("Loaded CONFIG and COSTS. Verify fee rates against your Groww contract notes.")


Loaded CONFIG and COSTS. Verify fee rates against your Groww contract notes.


In [9]:

import numpy as np
import pandas as pd
import yfinance as yf
from itertools import product

np.random.seed(42)

def _sma(series, n):
    return series.rolling(n, min_periods=n).mean()

def _ema(series, n):
    return series.ewm(span=n, adjust=False, min_periods=n).mean()

def _rsi(close, length=14):
    delta = close.diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=close.index).rolling(length).mean()
    roll_down = pd.Series(down, index=close.index).rolling(length).mean()
    rs = roll_up / roll_down.replace(0, np.nan)
    return 100 - (100 / (1 + rs))

def _macd(close, fast=12, slow=26, signal=9):
    ef = _ema(close, fast)
    es = _ema(close, slow)
    macd = ef - es
    sig = _ema(macd, signal)
    hist = macd - sig
    return macd, sig, hist

def _true_range(high, low, close):
    prev = close.shift(1)
    return pd.concat([high - low, (high - prev).abs(), (low - prev).abs()], axis=1).max(axis=1)

def _atr(h, l, c, n=14):
    return _true_range(h, l, c).rolling(n, min_periods=n).mean()

def CAGR(equity, freq_per_year=252):
    e = equity.dropna()
    if e.empty or e.iloc[0] <= 0:
        return 0.0
    years = len(e) / freq_per_year
    return (e.iloc[-1] / e.iloc[0]) ** (1.0 / years) - 1.0

def sharpe_ratio(returns, rf=0.0, freq_per_year=252):
    r = returns.dropna()
    if len(r) < 2 or r.std() == 0:
        return 0.0
    return float((r.mean() - rf / freq_per_year) / r.std() * np.sqrt(freq_per_year))

def max_drawdown(equity):
    roll_max = equity.cummax()
    dd = equity / roll_max - 1.0
    return float(dd.min()) if len(dd) else 0.0

def load_ohlc(ticker, start, end=None):
    df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False, multi_level_index=False)
    return df.dropna().rename(columns=str.title)


In [10]:

def _bps_to_frac(bps):
    return bps / 10000.0

def side_cost_rupees(notional, side, market_mode, COSTS):
    cfg = COSTS[market_mode]
    # Brokerage
    broker_fee = notional * _bps_to_frac(cfg["brokerage_bps"])
    cap = cfg["brokerage_cap_per_order"]
    if cap > 0:
        broker_fee = min(broker_fee, cap)
    broker_fee = max(broker_fee, cfg["brokerage_min"])
    # STT
    stt_bps = cfg["stt_buy_bps"] if side == "buy" else cfg["stt_sell_bps"]
    stt_fee = notional * _bps_to_frac(stt_bps)
    # Exchange & SEBI
    exch_fee = notional * _bps_to_frac(cfg["exchange_txn_bps"])
    sebi_fee = notional * _bps_to_frac(cfg["sebi_bps"])
    # Stamp
    stamp_bps = cfg["stamp_buy_bps"] if side == "buy" else cfg["stamp_sell_bps"]
    stamp_fee = notional * _bps_to_frac(stamp_bps)
    # GST on (brokerage + exchange)
    gst = (cfg["gst_pct"] / 100.0) * (broker_fee + exch_fee)
    return float(broker_fee + stt_fee + exch_fee + sebi_fee + stamp_fee + gst)


In [11]:

def build_features(df, params, use_ema=False):
    c, h, l, o = df["Close"], df["High"], df["Low"], df["Open"]
    fast, slow = params["fast_ma"], params["slow_ma"]
    rsi_len = params["rsi_len"]
    adx_len = params["adx_len"]
    atr_len = params["atr_len"]

    ma = _ema if use_ema else _sma
    df["MA_fast"] = ma(c, fast)
    df["MA_slow"] = ma(c, slow)
    df["RSI"] = _rsi(c, rsi_len)
    df["MACD"], df["MACD_signal"], df["MACD_hist"] = _macd(c, params["macd_fast"], params["macd_slow"], params["macd_signal"])
    df["ATR"] = _atr(h, l, c, atr_len)

    # ADX approx without TA-Lib
    up = h.diff()
    dn = -l.diff()
    plus_dm = np.where((up > dn) & (up > 0), up, 0.0)
    minus_dm = np.where((dn > up) & (dn > 0), dn, 0.0)
    tr = _true_range(h, l, c)
    atr_s = tr.rolling(adx_len, min_periods=adx_len).mean()
    plus_di = 100 * (pd.Series(plus_dm, index=df.index).rolling(adx_len).mean() / atr_s)
    minus_di = 100 * (pd.Series(minus_dm, index=df.index).rolling(adx_len).mean() / atr_s)
    dx = ((plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)) * 100
    df["ADX"] = dx.rolling(adx_len, min_periods=adx_len).mean()

    df.dropna(inplace=True)
    return df

def generate_signals(df, params, use_ema=False):
    df = build_features(df.copy(), params, use_ema=use_ema)
    trend_up = df["MA_fast"] > df["MA_slow"]
    confs = []
    if params["rsi_mode"] == "momentum":
        confs.append(df["RSI"] > params["rsi_mom_min"])
    elif params["rsi_mode"] == "mean_reversion":
        confs.append(df["RSI"] < params["rsi_mr_max"])
    if params["macd_mode"] == "above_signal_for_long":
        confs.append(df["MACD"] > df["MACD_signal"])
    if params["adx_min"] > 0:
        confs.append(df["ADX"] >= params["adx_min"])

    if confs:
        conf_all = confs[0].copy()
        for c in confs[1:]:
            conf_all = conf_all & c
    else:
        conf_all = pd.Series(True, index=df.index)

    df["LongEntrySignal"] = trend_up & conf_all
    df["LongExitSignal"] = df["MA_fast"] < df["MA_slow"]
    return df


In [12]:

def backtest_long_vol_sized(df, params, CONFIG, COSTS, capital=100000.0, phase="train", fold_id=0):
    slippage = CONFIG["slippage_bps"] / 10000.0
    market_mode = COSTS["market_mode"]
    stop_mult = params["stop_atr_mult"]
    rr_mult = params["tp_rr_multiple"]
    max_hold = params["max_hold_days"]

    df = df.copy()
    trades = []
    equity = pd.Series(index=df.index, dtype=float)
    cash = float(capital)
    position = 0
    entry_price = None
    entry_idx = None
    stop_price = None
    tp_price = None

    for i in range(1, len(df)):
        today = df.index[i]
        yest = df.index[i-1]

        # Exit first
        if position > 0:
            low, high = df.at[today, "Low"], df.at[today, "High"]
            exit_reason, exit_exec = None, None
            if low <= stop_price:
                exit_reason = "stop_loss"
                exit_exec = min(stop_price, df.at[today, "Open"]) * (1 - slippage)
            elif high >= tp_price:
                exit_reason = "take_profit"
                exit_exec = max(tp_price, df.at[today, "Open"]) * (1 - slippage)
            elif df.at[today, "LongExitSignal"]:
                exit_reason = "trend_break"
                exit_exec = df.at[today, "Open"] * (1 - slippage)
            elif max_hold and (i - entry_idx) >= max_hold:
                exit_reason = "max_hold"
                exit_exec = df.at[today, "Open"] * (1 - slippage)

            if exit_reason:
                notional_exit = position * exit_exec
                exit_costs = side_cost_rupees(notional_exit, "sell", market_mode, COSTS)
                proceeds = notional_exit - exit_costs
                entry_gross = position * entry_price
                entry_costs = side_cost_rupees(entry_gross, "buy", market_mode, COSTS)
                pnl = proceeds - (entry_gross + entry_costs)
                cash += proceeds
                trades.append({
                    "fold": fold_id, "phase": phase,
                    "entry_date": yest.strftime("%Y-%m-%d"),
                    "exit_date": today.strftime("%Y-%m-%d"),
                    "side": "LONG",
                    "entry_price": round(entry_price, 4),
                    "exit_price": round(exit_exec, 4),
                    "shares": int(position),
                    "reason": exit_reason,
                    "pnl": round(pnl, 2)
                })
                position = 0
                entry_price = None
                entry_idx = None
                stop_price = None
                tp_price = None

        # Entry next (use yesterday signal)
        if position == 0 and df.at[yest, "LongEntrySignal"]:
            atr = df.at[yest, "ATR"]
            if atr and not np.isnan(atr) and atr > 0:
                entry_px = df.at[today, "Open"] * (1 + slippage)
                stop_dist = stop_mult * atr
                if stop_dist > 0:
                    equity_now = cash
                    risk_rupees = equity_now * CONFIG["risk_per_trade_pct"]
                    est_shares = int(risk_rupees // stop_dist)
                    if est_shares > 0:
                        shares = est_shares
                        while shares > 0:
                            notional_entry = shares * entry_px
                            entry_costs = side_cost_rupees(notional_entry, "buy", market_mode, COSTS)
                            if notional_entry + entry_costs <= cash:
                                break
                            shares -= 1
                        if shares > 0:
                            cash -= (shares * entry_px + side_cost_rupees(shares * entry_px, "buy", market_mode, COSTS))
                            position = shares
                            entry_price = entry_px
                            entry_idx = i
                            stop_price = entry_px - stop_dist
                            tp_price = entry_px + rr_mult * stop_dist

        # Mark-to-market
        equity[today] = cash + (position * df.at[today, "Close"] if position > 0 else 0.0)

    # Force close at last bar
    if position > 0 and entry_price is not None:
        last = df.index[-1]
        exit_exec = df.at[last, "Close"] * (1 - slippage)
        notional_exit = position * exit_exec
        exit_costs = side_cost_rupees(notional_exit, "sell", market_mode, COSTS)
        proceeds = notional_exit - exit_costs
        entry_gross = position * entry_price
        entry_costs = side_cost_rupees(entry_gross, "buy", market_mode, COSTS)
        pnl = proceeds - (entry_gross + entry_costs)
        cash += proceeds
        trades.append({
            "fold": fold_id, "phase": phase,
            "entry_date": df.index[entry_idx-1].strftime("%Y-%m-%d") if entry_idx else "",
            "exit_date": last.strftime("%Y-%m-%d"),
            "side": "LONG",
            "entry_price": round(entry_price, 4),
            "exit_price": round(exit_exec, 4),
            "shares": int(position),
            "reason": "eod_close",
            "pnl": round(pnl, 2)
        })
        equity[last] = cash

    equity = equity.ffill().dropna()
    daily_returns = equity.pct_change().fillna(0.0)
    metrics = {
        "TotalReturn": float(equity.iloc[-1] / equity.iloc[0] - 1.0) if len(equity) else 0.0,
        "CAGR": float(CAGR(equity)),
        "Sharpe": float(sharpe_ratio(daily_returns)),
        "MaxDD": float(max_drawdown(equity)),
        "NumTrades": int(len(trades)),
        "FinalEquity": float(equity.iloc[-1]) if len(equity) else float(capital),
    }
    return pd.DataFrame(trades), equity, daily_returns, metrics


In [13]:

def param_grid_iter(grid):
    keys = list(grid.keys())
    for values in product(*(grid[k] for k in keys)):
        params = dict(zip(keys, values))
        if params["fast_ma"] >= params["slow_ma"]:
            continue
        yield params

def score_from_metrics(m, objective):
    return m["CAGR"] if objective == "CAGR" else m["TotalReturn"] if objective == "TotalReturn" else m["Sharpe"]

def make_folds(df, train_years=3, fwd_months=6, min_train_bars=504):
    idx = df.index
    if len(idx) < min_train_bars + 60:
        return []
    folds = []
    start = idx.min()
    end = idx.max()
    train_start = pd.to_datetime(start)
    while True:
        train_end = train_start + pd.DateOffset(years=train_years)
        fwd_end = train_end + pd.DateOffset(months=fwd_months)
        train = df.loc[(df.index >= train_start) & (df.index <= train_end)]
        fwd = df.loc[(df.index > train_end) & (df.index <= fwd_end)]
        if len(train) >= min_train_bars and len(fwd) >= 60:
            folds.append((train_start, train_end, fwd_end))
            train_start = train_start + pd.DateOffset(months=fwd_months)
            if fwd_end >= end - pd.Timedelta(days=5):
                break
        else:
            break
    return folds

def optimize_and_walkforward(ticker, df, CONFIG, COSTS):
    use_ema = CONFIG["use_ema"]
    objective = CONFIG["objective"]
    capital0 = CONFIG["initial_capital_per_ticker"]
    compound = CONFIG["compound_across_folds"]

    folds = make_folds(df, CONFIG["wf_train_years"], CONFIG["wf_forward_months"], CONFIG["min_train_bars"])
    if not folds:
        raise ValueError(f"{ticker}: not enough data to form walk-forward folds.")

    fwd_equity_all = []
    fwd_trades_all = []
    fold_param_rows = []
    current_capital = capital0

    for fi, (t_start, t_end, f_end) in enumerate(folds, start=1):
        train = df.loc[(df.index >= t_start) & (df.index <= t_end)].copy()
        fwd = df.loc[(df.index > t_end) & (df.index <= f_end)].copy()

        # Grid search on train
        best = {"score": -1e18, "params": None, "train_metrics": None}
        for params in param_grid_iter(CONFIG["grid"]):
            feats = generate_signals(train, params, use_ema=use_ema)
            trades, eq, rets, m = backtest_long_vol_sized(feats, params, CONFIG, COSTS, capital=capital0, phase="train", fold_id=fi)
            sc = score_from_metrics(m, objective)
            if sc > best["score"]:
                best = {"score": sc, "params": params, "train_metrics": m}

        # Forward with best
        feats_f = generate_signals(fwd, best["params"], use_ema=use_ema)
        start_cap = current_capital if compound else capital0
        trades_f, eq_f, rets_f, m_f = backtest_long_vol_sized(feats_f, best["params"], CONFIG, COSTS, capital=start_cap, phase="forward", fold_id=fi)
        current_capital = float(eq_f.iloc[-1])
        fwd_equity_all.append(eq_f)
        fwd_trades_all.append(trades_f)

        fold_param_rows.append({
            "Ticker": ticker, "Fold": fi,
            "Train_Start": t_start.strftime("%Y-%m-%d"),
            "Train_End": t_end.strftime("%Y-%m-%d"),
            "Fwd_End": f_end.strftime("%Y-%m-%d"),
            "BestParams": best["params"],
            "Train_NumTrades": best["train_metrics"]["NumTrades"],
            "Train_CAGR": round(best["train_metrics"]["CAGR"], 4),
            "Train_Sharpe": round(best["train_metrics"]["Sharpe"], 3),
        })

    # Aggregate forward
    if fwd_equity_all:
        eq_concat = pd.concat(fwd_equity_all).sort_index()
        eq_concat = eq_concat[~eq_concat.index.duplicated(keep="last")]
        daily_returns = eq_concat.pct_change().fillna(0.0)
        fwd_metrics = {
            "NumTrades": int(pd.concat(fwd_trades_all).shape[0]) if fwd_trades_all else 0,
            "TotalReturn": float(eq_concat.iloc[-1] / eq_concat.iloc[0] - 1.0),
            "CAGR": float(CAGR(eq_concat)),
            "Sharpe": float(sharpe_ratio(daily_returns)),
            "MaxDD": float(max_drawdown(eq_concat)),
            "FinalEquity": float(eq_concat.iloc[-1]),
        }
    else:
        fwd_metrics = {"NumTrades": 0, "TotalReturn": 0.0, "CAGR": 0.0, "Sharpe": 0.0, "MaxDD": 0.0, "FinalEquity": capital0}

    trades_all = pd.concat(fwd_trades_all, ignore_index=True) if fwd_trades_all else pd.DataFrame([])
    params_df = pd.DataFrame(fold_param_rows)
    return fwd_metrics, trades_all, params_df


In [14]:

summary_rows = []
for ticker in CONFIG["tickers"]:
    print("Processing", ticker, "...")
    try:
        df = load_ohlc(ticker, CONFIG["start_date"], CONFIG["end_date"])
        metrics, trades, params_df = optimize_and_walkforward(ticker, df, CONFIG, COSTS)
        tname = ticker.replace(".", "_")
        trades.to_csv(f"trades_{tname}.csv", index=False)
        params_df.to_csv(f"walkforward_params_{tname}.csv", index=False)
        summary_rows.append({
            "Ticker": ticker,
            "Fwd_NumTrades": metrics["NumTrades"],
            "Fwd_TotalReturn": round(metrics["TotalReturn"], 4),
            "Fwd_CAGR": round(metrics["CAGR"], 4),
            "Fwd_Sharpe": round(metrics["Sharpe"], 3),
            "Fwd_MaxDD": round(metrics["MaxDD"], 4),
            "Fwd_FinalEquity(INR)": round(metrics["FinalEquity"], 2),
        })
    except Exception as e:
        print("[WARN]", ticker, ":", e)

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv("optimized_results_summary.csv", index=False)
summary_df


Processing RELIANCE.NS ...
[WARN] RELIANCE.NS : single positional indexer is out-of-bounds
Processing TCS.NS ...
[WARN] TCS.NS : single positional indexer is out-of-bounds
Processing INFY.NS ...
[WARN] INFY.NS : single positional indexer is out-of-bounds
Processing HDFCBANK.NS ...
[WARN] HDFCBANK.NS : single positional indexer is out-of-bounds


""
